In [ ]:
!git clone https://github.com/airtlab/ConvLSTM4AD

#from google.colab import drive
#drive.mount('/content/drive')

Cloning into 'ConvLSTM4AD'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 471 (delta 12), reused 0 (delta 0), pack-reused 439
Receiving objects: 100% (471/471), 132.81 MiB | 60.90 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [ ]:
#!pip show tensorflow
#!pip install --upgrade tensorflow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn import model_selection
from sklearn.metrics import classification_report
import os
import nibabel as nib
import cv2
import numpy as np
!pip install simpleitk
import SimpleITK as sitk

     |████████████████████████████████| 47.4MB 158kB/s 


In [ ]:
img_path = "/content/ConvLSTM4AD/Merged_dataset/Dataset"
img_height, img_width = 256, 256 
seq_len = 5


# Read 
def read_img(img_path):
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path)) 

# Resample to 1 mm
def resample_img(itk_image, out_spacing=[1, 1, 1], is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)
    return resample.Execute(itk_image)

def preprocess(img, out_shape=None):
    if out_shape is not None:
        img = resize(img, out_shape, mode='constant')
    
    # Input normalization
    mean = img.mean() 
    std = img.std() 
    return (img - mean) / std

In [ ]:
X = []
Y = []

d = img_path
classes = ["AD","CN"]
classes_list = os.listdir(d) 
classes_list.sort()
print(classes_list[0]) 
print(classes_list[1]) 

# Sorting samples
full_path = os.path.join(d,classes_list[0]) # AD
samples = os.listdir(full_path)
samples.sort()
for p2 in samples: 
  p3 = os.path.join(full_path,p2) 
  my_img = nib.load(p3) 
  nii_data = my_img.get_fdata() 
  my_img.uncache()
  del my_img
  temp = []
  for i in range(0,5): 
    temp.append(cv2.resize(nii_data[:,:,i],(256,256)))
  X.append(temp)
  y = [0]*len(classes_list)
  y[1] = 1
  Y.append(y) # label [0,1]

full_path1 = os.path.join(d,classes_list[1]) # CN
samples1 = os.listdir(full_path1)
samples1.sort()
for p2 in samples1: 
  p3 = os.path.join(full_path1,p2) 
  my_img = nib.load(p3) 
  nii_data = my_img.get_fdata() 
  my_img.uncache()
  del my_img
  temp = []
  for i in range(0,5): 
    temp.append(cv2.resize(nii_data[:,:,i],(256,256)))
  X.append(temp)
  y = [0]*len(classes_list)
  y[0] = 1
  Y.append(y) # label [1,0] 

AD
CN


In [ ]:
X = np.asarray(X)
Y = np.asarray(Y)

X = X[...,np.newaxis]
print(X.shape)

(427, 5, 256, 256, 1)


In [ ]:
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X, Y, test_size = 0.20, shuffle = True, random_state = 0)

print("Number of training samples: " + str(len(X_train)))
print("Number of test samples: " + str(len(X_test)))

del X
del Y

Number of training samples: 341
Number of test samples: 86


In [ ]:
model = models.Sequential()

model.add(layers.ConvLSTM2D(filters=32, 
                            kernel_size=(3,3), 
                            return_sequences=False, 
                            data_format="channels_last", 
                            input_shape=(seq_len,img_height,img_width, 1)))
model.add(layers.Dropout(0.5)) 
model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(2, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 254, 254, 32)      38144     
_________________________________________________________________
dropout (Dropout)            (None, 254, 254, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 2064512)           0         
_________________________________________________________________
dense (Dense)                (None, 256)               528515328 
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 528,553,986
Trainable params: 528,553,986
Non-trainable params: 0
__________________________________________

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001) 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

earlystop = tf.keras.callbacks.EarlyStopping(patience=7)
callbacks = [earlystop]

history = model.fit(x=X_train, y=Y_train, epochs=40, batch_size=4, shuffle=True, validation_split=0.2, callbacks=callbacks) # training

Epoch 1/40
68/68 [==============================] - 140s 2s/step - loss: 2.7332 - accuracy: 0.6459 - val_loss: 0.4357 - val_accuracy: 0.8116
Epoch 2/40
68/68 [==============================] - 132s 2s/step - loss: 0.5120 - accuracy: 0.7520 - val_loss: 0.4318 - val_accuracy: 0.7971
Epoch 3/40
68/68 [==============================] - 131s 2s/step - loss: 0.3696 - accuracy: 0.8430 - val_loss: 0.3744 - val_accuracy: 0.7826
Epoch 4/40
68/68 [==============================] - 130s 2s/step - loss: 0.3333 - accuracy: 0.8506 - val_loss: 0.3744 - val_accuracy: 0.7826
Epoch 5/40
68/68 [==============================] - 129s 2s/step - loss: 0.2934 - accuracy: 0.8794 - val_loss: 0.3287 - val_accuracy: 0.7971
Epoch 6/40
68/68 [==============================] - 128s 2s/step - loss: 0.3438 - accuracy: 0.8411 - val_loss: 0.3377 - val_accuracy: 0.8261
Epoch 7/40
68/68 [==============================] - 126s 2s/step - loss: 0.3007 - accuracy: 0.8690 - val_loss: 0.3430 - val_accuracy: 0.7826
Epoch 8/40
68